In [1]:
import pandas as pd 
from functools import reduce
import string
# from spellchecker import SpellChecker
# spell = SpellChecker()
import nltk
porter = nltk.stem.PorterStemmer()
import numpy as np
import re
pd.options.mode.chained_assignment = None
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
#nltk.download('wordnet')
import math

In [2]:
user = pd.read_csv("data/c_user.csv")
tip = pd.read_csv("data/c_tip.csv")
rev = pd.read_csv("data/c_review.csv")
business = pd.read_csv("data/cinema.csv")

In [3]:
def append(x,y):
    return x+y
def clean(x):
    return list(filter(lambda x: False if x in stop_words or x == '' else True, x))
def word_process(s):
    s = s.translate(str.maketrans('', '', string.punctuation)).lower()
    s = re.sub("n't"," not", s)
    s = re.sub("n'"," not", s)
    return lem.lemmatize(s, "v") 

In [4]:
stop_words = pd.read_table("data/stop_words.txt")
stop_words = list(stop_words["words"])

In [5]:
# star = 5
# review = rev[rev["stars"] == star]
review = rev

In [6]:
word_init = list(map(lambda x: x.replace('\n', ' ').split(' '), list(review["text"])))

In [7]:
# sample = list(np.random.choice(len(word_init), size=100, replace=False))
# word = list(map(lambda x: list(map(word_process, x)), list(pd.Series(word_init)[sample])))
word = list(map(lambda x: list(map(word_process, x)), word_init))

In [8]:
word_clean = list(map(clean, word))

In [9]:
word_dict = {}
review_list = review
review_list["text"] = pd.Series(word_clean)
for i in list(review.index.values):
    data = review.loc[i]
    if type(data["text"]) == float:
        continue
    count = 1+data["useful"]+data["funny"]+data["cool"]
    time = (np.array('2019-01-01', dtype=np.datetime64)-np.array(review_list.loc[i]["date"].split(" ")[0], dtype=np.datetime64)).astype(dtype=np.float32)
    time_weight = 3-(time/365) if (3-(time/365))  > 0 else 0
    for w in review_list.loc[i]["text"]:
        if not w in word_dict:
            word_dict[w] = 0
        word_dict[w] += count*(0.8+time_weight)
word_dict

{'great': 70922.04657534482,
 'venue': 11667.48219178081,
 'seat': 172557.58356164224,
 'house': 9293.197260273986,
 'awesome': 13245.095890410856,
 'twice': 3377.1013698630177,
 'britney': 718.879452054794,
 'time': 102261.81643835889,
 'jlo': 465.32602739726025,
 'fantastic': 4781.843835616448,
 'performances': 1461.1205479452044,
 'sexy': 1390.7123287671232,
 'sultry': 10.038356164383561,
 'high': 9345.326027397272,
 'energy': 752.9205479452053,
 'love': 45730.424657534764,
 'recline': 32568.747945205232,
 'chair': 32705.789041095573,
 'arm': 4280.027397260276,
 'rest': 7261.208219178098,
 'rise': 1069.556164383562,
 'comfortable': 27529.35342465717,
 'movie': 214599.08219177363,
 'price': 49926.054794521166,
 'just': 82471.4054794537,
 'isnt': 11211.490410958913,
 'better': 26390.923287671023,
 'way': 35779.005479452026,
 'watch': 42560.4739726027,
 'sound': 25275.326027397095,
 'incredible': 2172.3945205479463,
 'picture': 7465.517808219195,
 'actually': 13542.013698630151,
 'trav

In [10]:
# count_filter = pd.Series(word_dict) >= sorted(list(word_dict.values()))[-50]
# count_filter = count_filter.reset_index()[0]
# pd.Series(list(word_dict.keys()))[count_filter]
def word_dict_fx(key):
    return word_dict[key]

In [11]:
hp_word = sorted(list(word_dict.keys()), key=word_dict_fx, reverse = True)
hp_word_count = pd.Series(word_dict)[hp_word]

In [12]:
import csv
w = csv.writer(open("output" + ".csv", "w"))
for key, val in hp_word_count.items():
    w.writerow([key, val])